In [1]:
import pandas as pd
import numpy as np
import pickle
from snownlp import SnowNLP
from tqdm.auto import tqdm
import re

In [2]:
detail = pd.read_excel('/home/afan/emotion_analysis/data/detail.xlsx')

In [4]:
detail

,_id,read_num,comment_num,title,detail_url,author,author_url,pub_time,list_url,id,name,code,type,page,pub_time_str,pub_time_int,content,content_images
0,5f3409810deff7cb2f5fb769,7286,61,2020年第一帖,"https://guba.eastmoney.com/news,gzqh,895422029...",期指一波流,NaN,01-01 00:00,NaN,4f8d4e395621ccb6d5dc98c9e2f474a3,股指期货吧,gzqh,2,194,2020-01-01,1577808000,含德之厚者，比于赤子。一年时光一晃就过，记得2019年第一帖就奉献了华丽的一章，虽然不是新帖...,[]
1,5f34617b689a8fdae489c176,1546,7,周四鸡蛋会跌停吗？,NaN,DQ皮卡丘,NaN,01-01 00:00,NaN,e3cfbc7dfa6a191599e9bbe50681a0a0,鸡蛋吧,jd,2,375,2020-01-01,1577808000,周四鸡蛋会跌停吗？,['https://gbres.dfcfw.com/Files/picture/202001...
2,5f3425d3689a8fdae483a584,3526,11,祝福慈文2020成为超级大牛股！,NaN,向太阳,NaN,01-01 00:00,NaN,b4515ea4e862c95afe46eaaf2d720d96,慈文传媒,002343,1,114,2020-01-01,1577808000,祝福慈文2020成为超级大牛股！东霞与骏马齐飞，慈文共华章一色。勇立潮头作品一流，一见惊鸿再...,['https://gbres.dfcfw.com/Files/picture/201912...
3,5f345978689a8fdae48900ca,1277,146,2020静待花开,NaN,悠然麓山,NaN,01-01 00:00,NaN,ac1b11e3b14c9a56be92522ff150ef7c,红塔证券,601236,1,395,2020-01-01,1577808000,2020静待花开,['https://gbres.dfcfw.com/Files/picture/202001...
4,5f345a8c689a8fdae4892463,3214,28,祝曙光的朋友们新年快乐，20年身体健康投资顺利,NaN,多头369,NaN,01-01 00:03,NaN,e72c70a8f5faccd4336e0fb9a76e5765,中科曙光,603019,1,348,2020-01-01,1577808000,祝曙光的朋友们新年快乐，20年身体健康投资顺利,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653198,5f345f53689a8fdae4899d45,3100,10,拼多多！多杀多！,NaN,畅游期货,NaN,02-24 15:19,NaN,09b78d80b483ba3551d6cc44c97a08c5,铁矿石吧,i,2,191,2020-02-24,1582473600,NaN,[]
653199,5f3460f6689a8fdae489afae,1062,4,真正的大支撑540，明显没到位，等着看申银呻吟，申银,NaN,操盘手k哥,NaN,02-06 11:33,NaN,c59dc301ac7318362c24b785b2ca538a,铁矿石吧,i,2,222,2020-02-06,1580918400,NaN,[]
653200,5f346122689a8fdae489b548,2018,16,最新重要资讯，持有者必看！,NaN,Hansen36999,NaN,02-27 17:14,NaN,7d88476c2bcf513af945b5cf8ba09237,德尔股份,300473,1,39,2020-02-27,1582732800,NaN,[]
653201,5f346138689a8fdae489b864,4461,17,哈哈！股市诡诈矣！解析今晚出的这个可能赔,NaN,股海无涯先识坑,NaN,04-30 23:41,NaN,540d020455f16bf1043676e7046b5ade,长电科技,600584,1,467,2020-04-30,1588176000,NaN,[]


In [11]:
id_df = detail.detail_url
id_df = pd.DataFrame(id_df.dropna())

id_list = []
for i in range(len(id_df)):
    row = id_df.iloc[i]
    d = row.detail_url
    id_list.append(d.split(',')[-1][:-5])

id_df['id'] = id_list
id_df

,detail_url,id
0,"https://guba.eastmoney.com/news,gzqh,895422029...",895422029
17,"https://guba.eastmoney.com/news,fczcema,895426...",895426940
77,"https://guba.eastmoney.com/news,300798,8954334...",895433400
95,"https://guba.eastmoney.com/news,300802,8954342...",895434281
100,"https://guba.eastmoney.com/news,gzqh,895434634...",895434634
...,...,...
652859,"https://guba.eastmoney.com/news,300601,9408751...",940875144
652860,"https://guba.eastmoney.com/news,300603,9429679...",942967974
652861,"https://guba.eastmoney.com/news,300498,9501365...",950136513
652862,"https://guba.eastmoney.com/news,300528,9075005...",907500578


In [84]:
len(id_df)

65520

In [95]:
id_df1 = id_df[['id']]
keys = list(id_df1.index)
values = list(id_df1.id)
# values = map(int,values)
post_dict = dict(zip(values,keys))
post_dict

{'895422029': 0,
 '895426940': 17,
 '895433400': 77,
 '895434281': 95,
 '895434634': 100,
 '895435649': 111,
 '895436918': 118,
 '895437972': 123,
 '895440105': 153,
 '895441090': 183,
 '895441202': 186,
 '895443846': 250,
 '895444966': 280,
 '895445545': 294,
 '895447047': 330,
 '895448635': 384,
 '895449435': 401,
 '895451348': 453,
 '895451788': 462,
 '895453184': 491,
 '895454334': 512,
 '895454421': 518,
 '895455935': 546,
 '895456419': 560,
 '895456436': 563,
 '895457018': 576,
 '895457996': 591,
 '895458906': 646,
 '895464771': 819,
 '895466653': 866,
 '895467930': 894,
 '895470015': 931,
 '895470276': 942,
 '895471057': 959,
 '895471109': 960,
 '895473338': 1018,
 '895474631': 1045,
 '895476707': 1086,
 '895476886': 1092,
 '895477622': 1107,
 '895477840': 1112,
 '895478914': 1133,
 '895479299': 1144,
 '895479413': 1148,
 '895480853': 1167,
 '895483035': 1210,
 '895484109': 1227,
 '895488680': 1286,
 '895488921': 1291,
 '895523706': 1386,
 '895566231': 1441,
 '895566419': 1442,


In [99]:
post_dict['896659164']

15853

In [17]:
detail['index'] = detail.index
detail_copy = detail[['index','content']]
detail_copy = detail_copy.dropna().reset_index(drop = True)
detail_copy

,index,content
0,0,含德之厚者，比于赤子。一年时光一晃就过，记得2019年第一帖就奉献了华丽的一章，虽然不是新帖...
1,1,周四鸡蛋会跌停吗？
2,2,祝福慈文2020成为超级大牛股！东霞与骏马齐飞，慈文共华章一色。勇立潮头作品一流，一见惊鸿再...
3,3,2020静待花开
4,4,祝曙光的朋友们新年快乐，20年身体健康投资顺利
...,...,...
642698,652792,说明价值投资才是上班族正确的投如果从我刚入股市时就一直持有美的，我的收益是6倍，说明价值投资...
642699,652793,买一手多，涨一个点，能赚多少钱
642700,652794,哎，割肉了，不碰它了
642701,652795,七位副总裁突击成股东来源：界面新闻 继创维集团(0751.HK)、比亚迪(002594.S...


In [105]:
detail_copy[detail_copy['index']==1046]

,index,content
1038,1046,有没有相信明天能涨3个点以上的集合！


In [103]:
post_partition_dict

642703

In [66]:
dic2[206152]

{'words': ['在',
  '全球',
  '股市',
  '暴跌',
  '和',
  'A',
  '股',
  '大幅',
  '下跌',
  '的',
  '背景',
  '下',
  '，',
  '前日',
  '见地',
  '量',
  '（24141',
  '万',
  '），',
  '今日',
  '现',
  '地价',
  '（5.35），',
  '两',
  '日',
  '累计',
  '跌幅',
  '仅',
  '2.1%（',
  '按',
  '收盘价',
  '计',
  '），',
  '表现',
  '出',
  '相当',
  '程度',
  '的',
  '抗跌',
  '性',
  '。',
  '本',
  '波',
  '调整',
  '幅度',
  '（7.51-6.35）',
  '约',
  '为',
  '前',
  '波',
  '反弹',
  '幅度',
  '（7.51-5.81）',
  '的',
  '三分之二',
  '，',
  '调整',
  '空间',
  '也',
  '基本',
  '到位',
  '。',
  '央行',
  '决定',
  '实施',
  '普惠',
  '金融',
  '定',
  '向',
  '降准',
  '，',
  '有助于',
  'A',
  '股',
  '开启',
  '反弹',
  '窗口',
  '。'],
 'emoji': []}

In [18]:
detail_copy[detail_copy['index']==278048]

,index,content
273275,278048,含量，有三个中科院院士坐镇。分享一下哥的分析。钢铁纳克是个好股非常有科技含量，有三个中科院院...


In [9]:
# 分词，筛选表情
emoji_pattern = re.compile(r'\[.\]|\[..\]|\[...\]|\[....\]')
d = {}
di_list = []
for i in tqdm(range(0,200000)):
    row = detail_copy.iloc[i]
    c = str(row.content)
    di = {}
    try:
        s = SnowNLP(c)
        w = s.words
    except:
        print('error1')
    try:
        di['words'] = w
    except:
        print('error2')
    emoji = emoji_pattern.findall(c) #从字符串中查找所有符合表情格式的内容
    emo = []
    for i in range(len(emoji)):
        it = emoji[i][1:-1]
        emo.append(it)
    try:
        di['emoji'] = emo
    except:
        print('error3')
    di_list.append(di)
inx_list = list(detail_copy['index'])[:200000]
d = dict(zip(inx_list,di_list))

# words_df = pd.DataFrame({'words': words})
# words_df['emoji'] = emojis
# words_df['index'] = detail_copy['index'][:200000]

  0%|          | 0/200000 [00:00<?, ?it/s]

In [10]:
with open("/home/afan/emotion_analysis/data/processed_data/post_dict1.p", "wb") as fp:   #Pickling
    pickle.dump(d, fp, protocol = pickle.HIGHEST_PROTOCOL)

In [54]:
with open("/home/afan/emotion_analysis/data/processed_data/post_dict1.p", "rb") as fp:   #Pickling
    dic1=pickle.load(fp)

In [55]:
with open("/home/afan/emotion_analysis/data/processed_data/post_dict2.p", "rb") as fp:   #Pickling
    dic2=pickle.load(fp) 

In [56]:
with open("/home/afan/emotion_analysis/data/processed_data/post_dict3.p", "rb") as fp:   #Pickling
    dic3=pickle.load(fp)

In [67]:
def add_dict(d1,d2):
    result = d1.copy()
    result.update(d2)
    return result


In [68]:
ans = add_dict(dic1, dic2)

In [69]:
words_dict = add_dict(ans, dic3)

In [70]:
with open("/home/afan/emotion_analysis/data/processed_data/post_dict.p", "wb") as fp:   #Pickling
    pickle.dump(words_dict, fp, protocol = pickle.HIGHEST_PROTOCOL)

In [71]:
with open("/home/afan/emotion_analysis/data/processed_data/post_dict.p", 'rb') as f:
    # {index: {'word':, 'emoji':}}
    post_partition_dict = pickle.load(f)

In [80]:
post_partition_dict[278048]

{'words': ['含量',
  '，',
  '有',
  '三',
  '个',
  '中科院',
  '院士',
  '坐镇',
  '。',
  '分享',
  '一下',
  '哥',
  '的',
  '分析',
  '。',
  '钢铁',
  '纳克',
  '是',
  '个',
  '好',
  '股',
  '非常',
  '有',
  '科技',
  '含量',
  '，',
  '有',
  '三',
  '个',
  '中科院',
  '院士',
  '坐镇',
  '。',
  '科技',
  '服务业',
  '，',
  '非常',
  '强大',
  '有',
  '实力',
  '呀',
  '！',
  '迷你',
  '盘',
  '。',
  '后市',
  '有',
  '很',
  '大',
  '的',
  '机会',
  '啊',
  '！'],
 'emoji': []}

In [78]:
detail_copy.iloc[399000]

index         406071
content    莫名其妙拉这么高。
Name: 399000, dtype: object

In [ ]:
detail_copy[detail_copy]